## Generate predictions on the test data set

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import itertools
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, accuracy_score
%matplotlib inline

In [5]:
# the model must be in the models folder
model_to_use = "model-004-0.82.h5"

In [6]:
from keras.models import load_model
model = load_model('models/'+model_to_use)